<a href="https://colab.research.google.com/github/jdilger/TensorFlowNotebooks/blob/master/ExampleEEificationAndDeploying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example notebook on loading a saved model and working through the steps to EE-ify the model and deplay to the AI platform for inferance in GEE.

There have been some minor updates since the first pass of notebooks we have used for this process which are reflected in the current example of hosting a [Hostable DNN for prediction in Earth Engine](https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb).



# Authentication steps

In [1]:
import os
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()


Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

# Load a saved model

Example loads a saved keras model in .h5 format. A TF dir could be loaded in the same manner as well.

Model src: 'gs://landfire/vgg16unet_model_jjd_mse_sigmoid.h5'

In [ ]:
MODEL_NAME = 'vgg16unet_model_jjd_mse_sigmoid.h5'
BUCKET = 'landfire'


model_path =  f"gs://{BUCKET}/{MODEL_NAME}"
# model.save(f"{MODEL_NAME}.h5")
new_model = tf.keras.models.load_model(model_path)

# Check its architecture
new_model.summary()

#Save the trained model

Export the trained model to TensorFlow SavedModel format in your cloud storage bucket. The Cloud Platform storage browser is useful for checking on these saved models. (taken directly from Hostable DNN for prediction in Earth Engine example).



Note: do we need to save it as an 'estimator' TF DIR? or could we keep it in the .h5 format?

 A. I think so... get_meta_graph_def(TF_FIR) reads the graph from a directory. 
 

In [10]:
#  save the model as a TF Estimator which is what 
MODEL_NAME_ESTIMATOR = MODEL_NAME.split('.')[0] + '_ESTIMATOR'
TF_DIR = 'gs://{}/{}'.format(BUCKET,MODEL_NAME_ESTIMATOR)
print(TF_DIR)
tf.keras.models.save_model(new_model,TF_DIR,save_format='tf')


gs://landfire/vgg16unet_model_jjd_mse_sigmoid_ESTIMATOR
INFO:tensorflow:Assets written to: gs://landfire/vgg16unet_model_jjd_mse_sigmoid_ESTIMATOR/assets


# EEification

EEIfication prepares the model for hosting on [Google AI Platform](https://cloud.google.com/ai-platform).  Learn more about EEification from [this doc](https://developers.google.com/earth-engine/tensorflow#interacting-with-models-hosted-on-ai-platform).  First, get (and SET) input and output names of the nodes.  **CHANGE THE OUTPUT NAME TO SOMETHING THAT MAKES SENSE FOR YOUR MODEL!**  Keep the input name of 'array', which is how you'll pass data into the model (as an array image).

In [11]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(TF_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
    input_name = v.name
    break

output_name = None
for k,v in outputs.items():
    output_name = v.name
    break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: 'pools2BP'}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input:0": "array"}'
'{"StatefulPartitionedCall:0": "pools2BP"}'


## Run the EEifier

The actual EEification is handled by the `earthengine model prepare` command.  Note that you will need to set your Cloud Project prior to running the command.

In [12]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://{}/eeified_{}/'.format(BUCKET,MODEL_NAME_ESTIMATOR)
# change to your specific project
PROJECT = 'pyregence-ee'
REGION = 'us-central1'
# # You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {TF_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://landfire/eeified_vgg16unet_model_jjd_mse_sigmoid_ESTIMATOR/' is ready to be hosted in AI Platform.


# Deploy and host the EEified model on AI Platform

Now there is another TensorFlow `SavedModel` stored in `EEIFIED_DIR` ready for hosting by AI Platform.  Do that from the `gcloud` command line tool, installed in the Colab runtime by default.  Be sure to specify a regional model with the `REGION` parameter.  Note that the `MODEL_NAME` must be unique.  If you already have a model by that name, either name a new model or a new version of the old model.  The [Cloud Console AI Platform models page](https://console.cloud.google.com/ai-platform/models) is useful for monitoring your models.

**If you change anything about the trained model, you'll need to re-EEify it and create a new version!**

In [ ]:
!gcloud ai-platform models create {MODEL_NAME_ESTIMATOR} --project {PROJECT} --region {REGION}

In [18]:
import time

MODEL_NAME = MODEL_NAME_ESTIMATOR
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

Creating version: v1647813426
Using endpoint [https://us-east1-ml.googleapis.com/]


In [23]:
print('MODEL_NAME=',MODEL_NAME)
print('VERSION_NAME=',VERSION_NAME)
print('PROJECT=',PROJECT)

MODEL_NAME= vgg16unet_model_jjd_mse_sigmoid_ESTIMATOR
VERSION_NAME= v1647813426
PROJECT= pyregence-ee
